In [8]:
import asyncio, boto3, pandas as pd, os, sys, pprint
pp = pprint.PrettyPrinter(indent=2, compact=True, width=80)

DAILY_UPLOADS_TABLE = "DailyUploadsTable-dev"
ddb = boto3.client("dynamodb", region_name="ap-south-1")

In [9]:
# import asyncio, concurrent.futures, functools
import concurrent.futures
# from multiprocessing import Pool
from subprocess import Popen, PIPE
import logging, pprint, os
from time import time

pp = pprint.PrettyPrinter(indent=2, compact=True, width=80)


class DownloadPosts:
    def __init__(self, subreddit, posts, logger):
        self.subreddit = subreddit
        self.posts = posts
        self.download_cmd = [
            "youtube-dl",
            "-f",
            "bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best",
            "placeholder_url",
            "-o",
            "placeholder_title",
        ]
        self.download_processes = []
        self.logger = logger
        self.download_processes_logs = []
        self.tasks = {}
        self.logger.info("Initialized object")

    def download_videos_from_posts(self):
        start = time()
        results = []
        with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
            futures = [executor.submit(download_video_from_post, post) for post in self.posts]
        
        for future in concurrent.futures.as_completed(futures):
            results.append(future.result())  
        end = time()
    
        self.logger.info("Results are:")
        self.logger.info(pp.pformat(results))

        self.logger.info(
            f"Time taken to download {len(self.posts)} videos from {self.subreddit} is {round(end-start, 2)}s"
        )


def download_video_from_post(post):
    logger = logging.getLogger()
    download_cmd = [
        "youtube-dl",
        "-f",
        "bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best",
        "placeholder_url",
        "-o",
        "placeholder_title",
    ]

    download_path = post["title"]
    download_cmd[3] = post["url"]
    download_cmd[5] = download_path

    process = Popen(download_cmd, stderr=PIPE)
    stdout, stderr = process.communicate()

    if stderr:
        logger.error("Failed to download the post:\n")
        logger.error(pp.pformat(post))
        logger.error("Failed with error")
        logger.error(pp.pformat(stderr.decode("utf-8")))
    else:
        logger.info("Successfully downloaded the post:\n")
        logger.info(pp.pformat(post))

# youtube-dl  command

In [10]:
# !  youtube-dl -f 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best' https://www.reddit.com/r/IdiotsInCars/comments/p4cm -o title

# !  youtube-dl -f 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best' https://www.reddit.com/r/IdiotsInCars/comments/p4cmjt/i_think_this_video_belongs_here/ -o title


In [11]:
import logging
posts = [
    {
        'url': 'https://www.reddit.com/r/IdiotsInCars/comments/p4cmjt/i_think_this_video_belongs_here/',
        'title': 'I think this belongs here'
    }   
]

# Initialize log config.
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()

In [12]:
download_posts = DownloadPosts(subreddit='funny', posts=posts, logger=logger)

INFO:root:Initialized object


In [13]:
download_posts.download_videos_from_posts()

[RedditR] p4cmjt: Downloading JSON metadata
[Reddit] nj6jgivg2dh71: Downloading m3u8 information
[Reddit] nj6jgivg2dh71: Downloading MPD manifest
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 5
[download] Destination: I think this belongs here.fhls-1530
[download] 100% of 4.86MiB in 00:01.97MiB/s ETA 00:00:40
[download] Destination: I think this belongs here.fdash-audio_0_132208
[download] 100% of 482.72KiB in 00:0054MiB/s ETA 00:001
[ffmpeg] Merging formats into "I think this belongs here.mp4"
Deleting original file I think this belongs here.fhls-1530 (pass -k to keep)
Deleting original file I think this belongs here.fdash-audio_0_132208 (pass -k to keep)


INFO:root:Successfully downloaded the post:

INFO:root:{ 'title': 'I think this belongs here',
  'url': 'https://www.reddit.com/r/IdiotsInCars/comments/p4cmjt/i_think_this_video_belongs_here/'}
INFO:root:Results are:
INFO:root:[None]
INFO:root:Time taken to download 1 videos from funny is 4.25s


In [15]:
item = { 'PK': {'S': '2021-08-16'},
'SK': {'S': 'funny'},
'posts': { 'L': [ { 'M': { 'author': {'S': 'SplungerPlunger'},
'name': {'S': 't3_p549q6'},
'title': {'S': 'my best goodwill find'},
'total_awards_received': {'N': '123'},
'ups': {'N': '73318'},
'upvote_ratio': {'N': '0.9'},
'url': {'S': 'https://v.redd.it/jjv7d2hdwlh71'}}},
{ 'M': { 'author': {'S': 'Mano_____'},
'name': {'S': 't3_p51s3k'},
'title': {'S': 'Don’t lose your focus.'},
'total_awards_received': {'N': '3'},
'ups': {'N': '1854'},
'upvote_ratio': {'N': '0.96'},
'url': {'S': 'https://v.redd.it/0vqji3ew6lh71'}}},
{ 'M': { 'author': {'S': 'sickwavesdude'},
'name': {'S': 't3_p5aze4'},
'title': { 'S': 'No ma’am, I don’t think I want '
'this stick up my nose'},
'total_awards_received': {'N': '4'},
'ups': {'N': '1616'},
'upvote_ratio': {'N': '0.92'},
'url': {'S': 'https://v.redd.it/m9wuwn1u5oh71'}}},
{ 'M': { 'author': {'S': 'RickHedgemints'},
'name': {'S': 't3_p5dxgr'},
'title': { 'S': 'In the bathroom in the dutch '
"amusment park ''Walibi'' there "
"is a ''party button'' when "
'pressed disco lights and '
'carnival music starts playing.'},
'total_awards_received': {'N': '2'},
'ups': {'N': '2532'},
'upvote_ratio': {'N': '0.97'},
'url': {'S': 'https://v.redd.it/tnilnv4jdph71'}}},
{ 'M': { 'author': {'S': 'Havocfyw'},
'name': {'S': 't3_p4z3lx'},
'title': { 'S': 'The Greatest Back Yard Wrestling '
'Entrance of All Time'},
'total_awards_received': {'N': '1'},
'ups': {'N': '653'},
'upvote_ratio': {'N': '0.93'},
'url': {'S': 'https://v.redd.it/1gmm2kphgkh71'}}},
{ 'M': { 'author': {'S': 'Ayuno-Tan'},
'name': {'S': 't3_p5deb6'},
'title': { 'S': 'popsicle is a hard word '
'sometimes...'},
'total_awards_received': {'N': '2'},
'ups': {'N': '831'},
'upvote_ratio': {'N': '0.91'},
'url': {'S': 'https://v.redd.it/bbcho3x96ph71'}}},
{ 'M': { 'author': {'S': 'Takkenwijf87'},
'name': {'S': 't3_p4uxpc'},
'title': { 'S': 'These guys are called KRPP or '
'Kiddy Ride Police Patrol and '
'ride around town. Checking up on '
'kiddy rides, scootmobiles and '
'cute pets. They have their own '
'soundtrack.'},
'total_awards_received': {'N': '0'},
'ups': {'N': '252'},
'upvote_ratio': {'N': '0.92'},
'url': {'S': 'https://v.redd.it/hmhtndidcjh71'}}},
{ 'M': { 'author': {'S': 'jodihas2kids'},
'name': {'S': 't3_p526ne'},
'title': {'S': 'Puppy vs door stop'},
'total_awards_received': {'N': '2'},
'ups': {'N': '221'},
'upvote_ratio': {'N': '0.91'},
'url': {'S': 'https://v.redd.it/c070ye4ualh71'}}},
{ 'M': { 'author': {'S': 'thistlegypsy'},
'name': {'S': 't3_p54vum'},
'title': { 'S': 'Difference between boys and '
'girls'},
'total_awards_received': {'N': '2'},
'ups': {'N': '197'},
'upvote_ratio': {'N': '0.77'},
'url': {'S': 'https://v.redd.it/lx1auqcy1mh71'}}},
{ 'M': { 'author': {'S': 'Yash817813'},
'name': {'S': 't3_p4y3ts'},
'title': { 'S': 'Penguin staring at the sunset 🌇 '
'🐧'},
'total_awards_received': {'N': '0'},
'ups': {'N': '139'},
'upvote_ratio': {'N': '0.88'},
'url': {'S': 'https://v.redd.it/pjlafy467kh71'}}},
{ 'M': { 'author': {'S': 'NoodleAstronaut'},
'name': {'S': 't3_p5c76p'},
'title': {'S': 'Let me see your Indian war cry!'},
'total_awards_received': {'N': '0'},
'ups': {'N': '254'},
'upvote_ratio': {'N': '0.86'},
'url': {'S': 'https://v.redd.it/tzlisrhmooh71'}}},
{ 'M': { 'author': {'S': 'micheleberaudo'},
'name': {'S': 't3_p4zxht'},
'title': {'S': 'Just a normal camping day'},
'total_awards_received': {'N': '0'},
'ups': {'N': '69'},
'upvote_ratio': {'N': '0.78'},
'url': {'S': 'https://v.redd.it/bbbyix1rokh71'}}},
{ 'M': { 'author': {'S': 'fryether'},
'name': {'S': 't3_p563ud'},
'title': { 'S': 'Apparently my son understands a '
'practical joke at 11 months'},
'total_awards_received': {'N': '1'},
'ups': {'N': '63'},
'upvote_ratio': {'N': '0.71'},
'url': {'S': 'https://v.redd.it/0ulzinopgmh71'}}},
{ 'M': { 'author': {'S': 'JehovasHitMan'},
'name': {'S': 't3_p4v376'},
'title': { 'S': 'Now a lawyers gonna get '
'involved.'},
'total_awards_received': {'N': '0'},
'ups': {'N': '55'},
'upvote_ratio': {'N': '0.7'},
'url': {'S': 'https://v.redd.it/5227hzy1ejh71'}}},
{ 'M': { 'author': {'S': 'Additional_Plant_539'},
'name': {'S': 't3_p5fx9s'},
'title': {'S': 'Wall To Wall Raccoons'},
'total_awards_received': {'N': '3'},
'ups': {'N': '195'},
'upvote_ratio': {'N': '0.93'},
'url': {'S': 'https://v.redd.it/uvn691wl0qh71'}}},
{ 'M': { 'author': {'S': 'Asleep-Bat618'},
'name': {'S': 't3_p5c1vq'},
'title': {'S': 'DOOM'},
'total_awards_received': {'N': '0'},
'ups': {'N': '61'},
'upvote_ratio': {'N': '0.81'},
'url': {'S': 'https://v.redd.it/h0zj6rtamoh71'}}},
{ 'M': { 'author': {'S': 'BeTTish_09'},
'name': {'S': 't3_p4ymet'},
'title': { 'S': 'Wait for the MOM ..and her '
'laugh!🤣🤣'},
'total_awards_received': {'N': '0'},
'ups': {'N': '59'},
'upvote_ratio': {'N': '0.71'},
'url': {'S': 'https://v.redd.it/pa8eepnzbkh71'}}},
{ 'M': { 'author': {'S': 'TimHamburg'},
'name': {'S': 't3_p4ui6t'},
'title': { 'S': 'Round Sunglasses are for '
'Villains: Part 2'},
'total_awards_received': {'N': '2'},
'ups': {'N': '45'},
'upvote_ratio': {'N': '0.79'},
'url': {'S': 'https://v.redd.it/ubfn2mw48jh71'}}},
{ 'M': { 'author': {'S': 'Zombyte701'},
'name': {'S': 't3_p588s7'},
'title': {'S': '*Fortunate son intensifies*'},
'total_awards_received': {'N': '0'},
'ups': {'N': '38'},
'upvote_ratio': {'N': '0.64'},
'url': {'S': 'https://v.redd.it/3of91poy4nh71'}}},
{ 'M': { 'author': {'S': 'Kawamasu'},
'name': {'S': 't3_p5d4ue'},
'title': {'S': "Help I'm stuck!"},
'total_awards_received': {'N': '0'},
'ups': {'N': '44'},
'upvote_ratio': {'N': '0.75'},
'url': {'S': 'https://v.redd.it/klcr06fe2ph71'}}},
{ 'M': { 'author': {'S': 'ErockLobster'},
'name': {'S': 't3_p5270o'},
'title': { 'S': 'Mission ImPAWSable (original '
'credit to u/pazluz)'},
'total_awards_received': {'N': '0'},
'ups': {'N': '36'},
'upvote_ratio': {'N': '0.77'},
'url': {'S': 'https://v.redd.it/yqpwfe0nalh71'}}},
{ 'M': { 'author': {'S': 'Havocfyw'},
'name': {'S': 't3_p4wt0t'},
'title': {'S': "Quick, she's coming! Act casual."},
'total_awards_received': {'N': '0'},
'ups': {'N': '34'},
'upvote_ratio': {'N': '0.73'},
'url': {'S': 'https://v.redd.it/zamfhd9iujh71'}}},
{ 'M': { 'author': {'S': 'PavlovsCat333'},
'name': {'S': 't3_p56gqh'},
'title': { 'S': 'Randy raccoons cavorting in my '
'yard'},
'total_awards_received': {'N': '1'},
'ups': {'N': '35'},
'upvote_ratio': {'N': '0.89'},
'url': {'S': 'https://v.redd.it/6kvgjphkkmh71'}}},
{ 'M': { 'author': {'S': 'teyko17'},
'name': {'S': 't3_p4u6pq'},
'title': {'S': 'A bugs life'},
'total_awards_received': {'N': '0'},
'ups': {'N': '33'},
'upvote_ratio': {'N': '0.79'},
'url': {'S': 'https://v.redd.it/ykv0v8hw4jh71'}}},
{ 'M': { 'author': {'S': 'BostonFan69'},
'name': {'S': 't3_p4wmra'},
'title': {'S': 'Supr Smsh Brs Spngbb'},
'total_awards_received': {'N': '0'},
'ups': {'N': '27'},
'upvote_ratio': {'N': '0.74'},
'url': {'S': 'https://v.redd.it/y0h75ne3tjh71'}}},
{ 'M': { 'author': {'S': 'sjstirl'},
'name': {'S': 't3_p59ys1'},
'title': {'S': 'Bonk'},
'total_awards_received': {'N': '2'},
'ups': {'N': '30'},
'upvote_ratio': {'N': '0.64'},
'url': {'S': 'https://v.redd.it/qwer8fccrnh71'}}},
{ 'M': { 'author': {'S': 'gokuorkakarot'},
'name': {'S': 't3_p4xuj4'},
'title': {'S': 'Its the pig from loonie toons'},
'total_awards_received': {'N': '1'},
'ups': {'N': '22'},
'upvote_ratio': {'N': '0.65'},
'url': {'S': 'https://v.redd.it/atb1ud9o4kh71'}}},
{ 'M': { 'author': {'S': 'itsavism'},
'name': {'S': 't3_p5bc0l'},
'title': { 'S': 'Hi All! An irrelevant fact - '
'Kevin, Dwight and Nard Dog are '
'my favourite.'},
'total_awards_received': {'N': '1'},
'ups': {'N': '17'},
'upvote_ratio': {'N': '0.78'},
'url': {'S': 'https://v.redd.it/l7qjmmnzaoh71'}}},
{ 'M': { 'author': {'S': 'caswunn'},
'name': {'S': 't3_p4vrir'},
'title': { 'S': 'Did you know that giraffes are '
'ticklish?'},
'total_awards_received': {'N': '0'},
'ups': {'N': '13'},
'upvote_ratio': {'N': '0.71'},
'url': {'S': 'https://v.redd.it/nish3svnkjh71'}}},
{ 'M': { 'author': {'S': 'ItIsMathewFrom2003'},
'name': {'S': 't3_p5b0fo'},
'title': { 'S': 'Sad version of Happy Birthday '
'song.'},
'total_awards_received': {'N': '0'},
'ups': {'N': '13'},
'upvote_ratio': {'N': '0.72'},
'url': {'S': 'https://v.redd.it/by7cig976oh71'}}},
{ 'M': { 'author': {'S': 'iammisaac'},
'name': {'S': 't3_p58d3r'},
'title': {'S': 'She was excited to game with me'},
'total_awards_received': {'N': '0'},
'ups': {'N': '9'},
'upvote_ratio': {'N': '0.57'},
'url': {'S': 'https://v.redd.it/4ec6b0kk6nh71'}}},
{ 'M': { 'author': {'S': 'Havocfyw'},
'name': {'S': 't3_p4ucc8'},
'title': { 'S': 'Ancient contraceptive '
'technique... It takes a village'},
'total_awards_received': {'N': '0'},
'ups': {'N': '9'},
'upvote_ratio': {'N': '0.59'},
'url': {'S': 'https://v.redd.it/e5aojtkx5jh71'}}},
{ 'M': { 'author': {'S': 'GreetingsFromAP'},
'name': {'S': 't3_p4xe2b'},
'title': { 'S': 'I know you are, but what am I? '
'Reminiscence trailer looks ok, '
'but the casting is spot on.'},
'total_awards_received': {'N': '0'},
'ups': {'N': '5'},
'upvote_ratio': {'N': '0.66'},
'url': {'S': 'https://v.redd.it/egk9tqf3zjh71'}}},
{ 'M': { 'author': {'S': 'porygonque_19'},
'name': {'S': 't3_p5g5v7'},
'title': {'S': '“Baby up in this B*tch'},
'total_awards_received': {'N': '0'},
'ups': {'N': '1'},
'upvote_ratio': {'N': '0.54'},
'url': {'S': 'https://v.redd.it/41pa06q23qh71'}}},
{ 'M': { 'author': {'S': 'bryce_engineer'},
'name': {'S': 't3_p586sj'},
'title': {'S': 'Poor Psyduck.'},
'total_awards_received': {'N': '0'},
'ups': {'N': '1'},
'upvote_ratio': {'N': '0.52'},
'url': { 'S': 'https://v.redd.it/wrgxtj354nh71'}}}]}}


In [16]:
from entities.GatherPosts import GatherPosts
from entities.DownloadPosts import DownloadPosts

In [17]:
gather_posts = GatherPosts(subreddit="funny", logger=logging.getLogger())

In [22]:
deserialized_item = GatherPosts.deserialize_from_item(item)

In [26]:
posts = deserialized_item['posts']
download_posts = DownloadPosts(subreddit='funny', posts = posts, logger=logging.getLogger())

INFO:root:Initialized object


In [27]:
download_posts.download_videos_from_posts()

[Reddit] m9wuwn1u5oh71: Downloading m3u8 information
[Reddit] tnilnv4jdph71: Downloading m3u8 information
[Reddit] jjv7d2hdwlh71: Downloading m3u8 information
[Reddit] 0vqji3ew6lh71: Downloading m3u8 information
[Reddit] 1gmm2kphgkh71: Downloading m3u8 information
[Reddit] tnilnv4jdph71: Downloading MPD manifest
[Reddit] m9wuwn1u5oh71: Downloading MPD manifest
[Reddit] 1gmm2kphgkh71: Downloading MPD manifest
[Reddit] 0vqji3ew6lh71: Downloading MPD manifest
[Reddit] jjv7d2hdwlh71: Downloading MPD manifest
[download] Destination: my best goodwill find
[download]  27.2% of 1.84MiB at  3.24MiB/s ETA 00:00known ETA

INFO:root:Successfully downloaded the post:

INFO:root:{ 'author': 'SplungerPlunger',
  'name': 't3_p549q6',
  'title': 'my best goodwill find',
  'total_awards_received': 123,
  'ups': 73318,
  'upvote_ratio': 0.9,
  'url': 'https://v.redd.it/jjv7d2hdwlh71'}


[download] 100% of 1.84MiB in 00:0082MiB/s ETA 00:00
[Reddit] bbcho3x96ph71: Downloading m3u8 information
[Reddit] bbcho3x96ph71: Downloading MPD manifest
[download] Destination: popsicle is a hard word sometimes....fdash-video_2124697
[download] 100% of 2.67MiB in 00:0071MiB/s ETA 00:00known ETA


INFO:root:Successfully downloaded the post:

INFO:root:{ 'author': 'Ayuno-Tan',
  'name': 't3_p5deb6',
  'title': 'popsicle is a hard word sometimes...',
  'total_awards_received': 2,
  'ups': 831,
  'upvote_ratio': 0.91,
  'url': 'https://v.redd.it/bbcho3x96ph71'}


[download] Destination: popsicle is a hard word sometimes....fdash-audio_0_131323
[download] 100% of 169.42KiB in 00:0040MiB/s ETA 00:00:00wn ETA
[ffmpeg] Merging formats into "popsicle is a hard word sometimes....mp4"
Deleting original file popsicle is a hard word sometimes....fdash-video_2124697 (pass -k to keep)
Deleting original file popsicle is a hard word sometimes....fdash-audio_0_131323 (pass -k to keep)
[Reddit] hmhtndidcjh71: Downloading m3u8 information
[Reddit] hmhtndidcjh71: Downloading MPD manifest
[download] Destination: These guys are called KRPP or Kiddy Ride Police Patrol and ride around town. Checking up on kiddy rides, scootmobiles and cute pets. They have their own soundtrack..fdash-video_2272722
[download] 100% of 6.34MiB in 00:0159MiB/s ETA 00:00known ETA
[download] Destination: These guys are called KRPP or Kiddy Ride Police Patrol and ride around town. Checking up on kiddy rides, scootmobiles and cute pets. They have their own soundtrack..fdash-audio_0_132250
[

...

[download] Destination: “Baby up in this B*tch.fdash-video_2456635
[download] 100% of 2.67MiB in 00:0070MiB/s ETA 00:00known ETA


INFO:root:Successfully downloaded the post:

INFO:root:{ 'author': 'porygonque_19',
  'name': 't3_p5g5v7',
  'title': '“Baby up in this B*tch',
  'total_awards_received': 0,
  'ups': 1,
  'upvote_ratio': 0.54,
  'url': 'https://v.redd.it/41pa06q23qh71'}


[download] Destination: “Baby up in this B*tch.fdash-audio_0_133738
[download] 100% of 149.60KiB in 00:0023MiB/s ETA 00:00:00wn ETA
[ffmpeg] Merging formats into "“Baby up in this B*tch.mp4"
Deleting original file “Baby up in this B*tch.fdash-video_2456635 (pass -k to keep)
Deleting original file “Baby up in this B*tch.fdash-audio_0_133738 (pass -k to keep)
[download] Destination: I know you are, but what am I? Reminiscence trailer looks ok, but the casting is spot on..fdash-video_1034695
[download] 100% of 4.31MiB in 00:1130MiB/s ETA 00:005
[download] Destination: I know you are, but what am I? Reminiscence trailer looks ok, but the casting is spot on..fdash-audio_0_129978
[download]  45.8% of 556.94KiB at  5.28MiB/s ETA 00:000

INFO:root:Successfully downloaded the post:

INFO:root:{ 'author': 'GreetingsFromAP',
  'name': 't3_p4xe2b',
  'title': 'I know you are, but what am I? Reminiscence trailer looks ok, but '
           'the casting is spot on.',
  'total_awards_received': 0,
  'ups': 5,
  'upvote_ratio': 0.66,
  'url': 'https://v.redd.it/egk9tqf3zjh71'}
INFO:root:Results are:
INFO:root:[ None, None, None, None, None, None, None, None, None, None, None, None, None,
  None, None, None, None, None, None, None, None, None, None, None, None, None,
  None, None, None, None, None, None, None, None, None]
INFO:root:Time taken to download 35 videos from funny is 51.13s


[download] 100% of 556.94KiB in 00:0038MiB/s ETA 00:00
[ffmpeg] Merging formats into "I know you are, but what am I? Reminiscence trailer looks ok, but the casting is spot on..mp4"
Deleting original file I know you are, but what am I? Reminiscence trailer looks ok, but the casting is spot on..fdash-video_1034695 (pass -k to keep)
Deleting original file I know you are, but what am I? Reminiscence trailer looks ok, but the casting is spot on..fdash-audio_0_129978 (pass -k to keep)


In [15]:
import youtube_dl, logging

# bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best
ydl_opts = {    

    'format' : 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best',
    'logger': logging.getLogger(),   
    'outtmpl': 'hello'
}

# ydl_opts = {
#     'format': 'bestaudio/best',
#     'postprocessors': [{
#         'key': 'FFmpegExtractAudio',
#         'preferredcodec': 'wav',
#         'preferredquality': '192'
#     }],
#     'postprocessor_args': [
#         '-ar', '16000'
#     ],
#     'prefer_ffmpeg': True,
#     'keepvideo': True
# }

In [16]:
with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    ydl.download(['https://www.reddit.com/r/IdiotsInCars/comments/p4cmjt/i_think_this_video_belongs_here/'])

# New implementation with youtube-dl package

In [50]:
# import asyncio, concurrent.futures, functools
# import concurrent.futures

from multiprocessing import Process, Pipe

# from subprocess import Popen, PIPE
import logging, pprint, os, youtube_dl
from time import time

pp = pprint.PrettyPrinter(indent=2, compact=True, width=80)


class DownloadPosts:
    def __init__(self, subreddit, posts, logger):
        self.subreddit = subreddit
        self.posts = posts
        self.download_cmd = [
            "youtube-dl",
            "-f",
            "bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best",
            "placeholder_url",
            "-o",
            "placeholder_title",
        ]
        self.download_processes = []
        self.logger = logger
        self.download_processes_logs = []
        self.tasks = {}

    def __download_video_from_post(self, post, conn, logger):
        logger.info(
            f"Process: {os.getpid()} is downloading video from post with title: {post['title']}"
        )
        ydl_opts = {
            "format": "bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best",
            "logger": logging.getLogger(),
            "outtmpl": "/tmp/" + post['title'],
        }

        with youtube_dl.YoutubeDL(ydl_opts) as ydl:
            ydl.download([post["url"]])

        conn.send("done")
        conn.close()
        logger.info(
            f"Process: {os.getpid()} finished downloading and is shutting down."
        )

    def download_videos_from_posts(self):
        self.logger.info("Downloading videos from posts")
        parent_connections = []
        processes = []
        posts_completed = []
        start = time()
        for post in self.posts:
            parent_conn, child_conn = Pipe()
            parent_connections.append(parent_conn)

            process = Process(
                target=self.__download_video_from_post,
                args=(post, child_conn, self.logger),
            )
            processes.append(process)

        for process in processes:
            process.start()

        for process in processes:
            process.join()

        for parent_conn in parent_connections:
            posts_completed.append(parent_conn.recv())

        end = time()

        self.logger.info("Received the following messages from child processes")
        self.logger.info(pp.pformat(posts_completed))
        self.logger.info(
            f"Finished processing {posts_completed} in {round(end - start, 2)} seconds"
        )

In [51]:
import logging
posts = [
    {
        'url': 'https://www.reddit.com/r/IdiotsInCars/comments/p4cmjt/i_think_this_video_belongs_here/',
        'title': 'I think this belongs here'
    },
    {
        'url': 'https://www.reddit.com/r/IdiotsInCars/comments/p5mjbv/just_a_mustang_doing_mustang_things/',
        'title': 'Just a Mustang doing Mustang things'
    },
    {
        'url': 'https://www.reddit.com/r/therewasanattempt/comments/p5ismm/to_hop_on_a_bus/',
        'title': 'To hop on a bus'
    }
]

# Initialize log config.
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()


In [52]:
download_posts = DownloadPosts(subreddit='funny', posts = posts, logger=logger)

In [53]:
download_posts.download_videos_from_posts()

INFO:root:Downloading videos from posts
INFO:root:Process: 19105 is downloading video from post with title: I think this belongs here
INFO:root:Process: 19106 is downloading video from post with title: Just a Mustang doing Mustang things
INFO:root:Process: 19111 is downloading video from post with title: To hop on a bus
INFO:root:Process: 19111 finished downloading and is shutting down.
INFO:root:Process: 19105 finished downloading and is shutting down.
INFO:root:Process: 19106 finished downloading and is shutting down.
INFO:root:Received the following messages from child processes
INFO:root:['done', 'done', 'done']
INFO:root:Finished processing ['done', 'done', 'done'] in 7.08 seconds


In [2]:
import boto3

In [3]:
s3 = boto3.client('s3')

In [58]:
resp = s3.create_bucket(
    ACL="private",
    Bucket="whodoesntlovereddit-bla",
    CreateBucketConfiguration={
        'LocationConstraint': 'ap-south-1'
    }
)

In [59]:
resp

{'ResponseMetadata': {'RequestId': 'SZ5D0FHWRJPVNK5N',
  'HostId': 'SyR0CJaoGAm0OZcbFqL4MZqQDqeXxL6PLimAza93vG8rixk9VBOY/+V75SkQEzJyk/OQTX2y/Rc=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'SyR0CJaoGAm0OZcbFqL4MZqQDqeXxL6PLimAza93vG8rixk9VBOY/+V75SkQEzJyk/OQTX2y/Rc=',
   'x-amz-request-id': 'SZ5D0FHWRJPVNK5N',
   'date': 'Tue, 17 Aug 2021 10:14:50 GMT',
   'location': 'http://whodoesntlovereddit-bla.s3.amazonaws.com/',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'Location': 'http://whodoesntlovereddit-bla.s3.amazonaws.com/'}

In [61]:
resp = s3.put_object(
    Bucket='whodoesntlovereddit-bla',
    Body='./requirements.txt',
    Key='requirements.txt'
)

In [17]:
s3.head_bucket(Bucket='whodoesntlovereddit-2021-08-17-funny')

ClientError: An error occurred (404) when calling the HeadBucket operation: Not Found

In [6]:
s3.head_bucket(Bucket='whodoesntlovereddit-2021-08-17-funn')

ClientError: An error occurred (404) when calling the HeadBucket operation: Not Found

In [16]:
try:
    s3.head_bucket(Bucket='whodoesntlovereddit-2021-08-17-funny')
except:
    print("hello")

hello


In [12]:
file_name = 'bla.mp4'
file_path = './' + file_name
with open(file_path, "rb") as f:
            s3.upload_fileobj(f, 'whodoesntlovereddit-2021-08-17-funny', file_name)